In [67]:
import numpy as np
import pandas as pd
from scipy import stats
 
# グラフ描画
from matplotlib import pylab as plt
import seaborn as sns
%matplotlib inline

In [68]:
f3 = '%Y/%m/%d %H:%M'
my_parser = lambda date: pd.datetime.strptime(date, f3)

In [69]:
df1 = pd.read_csv('mid1.csv',parse_dates=True,date_parser=my_parser)
df1.head(5)

,btcdate,btcbegin,btchigh,btclow,btcend,cboebegin,cboehigh,cboelow,cboeend,cboevolumn
0,2017-12-11 07:00:00,14690.4258,15184.1602,14538.3965,14646.6904,14710.0,14710.0,14710.0,14710.0,1.0
1,2017-12-11 08:00:00,14646.6904,15916.0957,14643.7852,15097.2529,15570.0,15755.0,15570.0,15755.0,1.5
2,2017-12-11 09:00:00,15097.2529,15546.0771,15060.0107,15459.7676,16430.0,16800.0,16430.0,16800.0,2.0
3,2017-12-11 10:00:00,15459.7676,16179.0859,15458.7402,16162.8467,17000.0,17260.0,17000.0,17260.0,3.0
4,2017-12-11 11:00:00,16162.8467,16503.3926,16004.5996,16502.0645,17540.0,18660.0,17400.0,18660.0,7.0


In [70]:
#normalization
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(df1.drop(['btcdate'],axis=1)) #do not need date column
x_scaled[:5]
x_scaled.shape

(1220, 9)

In [71]:
df2 = pd.DataFrame(x_scaled)
df2.columns = df1.drop(['btcdate'],axis=1).columns
df2.head(5)


,btcbegin,btchigh,btclow,btcend,cboebegin,cboehigh,cboelow,cboeend,cboevolumn
0,0.650832,0.674454,0.659420,0.647569,0.619971,0.593113,0.628854,0.621563,0.000000
1,0.647569,0.729524,0.667486,0.681184,0.681753,0.666550,0.690521,0.697178,0.000699
2,0.681184,0.701684,0.699340,0.708230,0.743534,0.739986,0.752187,0.772793,0.001399
3,0.708230,0.749311,0.729855,0.760685,0.784483,0.772312,0.793059,0.806078,0.002797
4,0.760685,0.773711,0.771630,0.785993,0.823276,0.870696,0.821741,0.907381,0.008392


In [72]:
#now we need add plus/minus flag columns with time lag
ser3h = df2['btcbegin'].diff(periods=3)
ser12h = df2['btcbegin'].diff(periods=12)
ser1d = df2['btcbegin'].diff(periods=24)
ser2d = df2['btcbegin'].diff(periods=48)

ser3hafter = df2['btcbegin'].diff(periods=-3)
ser12hafter = df2['btcbegin'].diff(periods=-12)
ser1dafter = df2['btcbegin'].diff(periods=-24)
ser2dafter = df2['btcbegin'].diff(periods=-48)


In [73]:
# add flags to dataframe
df2['3hbefore'] = ser3h
df2['12hbefore'] = ser12h
df2['1dbefore'] = ser1d
df2['2dbefore'] = ser2d

df2['3hafter'] = ser3hafter
df2['12hafter'] = ser12hafter
df2['1dafter'] = ser1dafter
df2['2dafter'] = ser2dafter

df2.head(5)

,btcbegin,btchigh,btclow,btcend,cboebegin,cboehigh,cboelow,cboeend,cboevolumn,3hbefore,12hbefore,1dbefore,2dbefore,3hafter,12hafter,1dafter,2dafter
0,0.650832,0.674454,0.659420,0.647569,0.619971,0.593113,0.628854,0.621563,0.000000,NaN,NaN,NaN,NaN,-0.057398,-0.147411,-0.183366,-0.190829
1,0.647569,0.729524,0.667486,0.681184,0.681753,0.666550,0.690521,0.697178,0.000699,NaN,NaN,NaN,NaN,-0.113115,-0.152347,-0.171773,-0.200015
2,0.681184,0.701684,0.699340,0.708230,0.743534,0.739986,0.752187,0.772793,0.001399,NaN,NaN,NaN,NaN,-0.104808,-0.103843,-0.123340,-0.153075
3,0.708230,0.749311,0.729855,0.760685,0.784483,0.772312,0.793059,0.806078,0.002797,0.057398,NaN,NaN,NaN,-0.070515,-0.083014,-0.095700,-0.147952
4,0.760685,0.773711,0.771630,0.785993,0.823276,0.870696,0.821741,0.907381,0.008392,0.113115,NaN,NaN,NaN,-0.012753,-0.024061,-0.057727,-0.069803


In [74]:
df2.tail(5)

,btcbegin,btchigh,btclow,btcend,cboebegin,cboehigh,cboelow,cboeend,cboevolumn,3hbefore,12hbefore,1dbefore,2dbefore,3hafter,12hafter,1dafter,2dafter
1215,0.318581,0.306976,0.324008,0.323278,0.298851,0.284610,0.305464,0.297395,0.141259,0.027465,0.037789,0.063747,0.122739,0.008613,NaN,NaN,NaN
1216,0.323278,0.307016,0.324910,0.315324,0.298851,0.280394,0.303313,0.295224,0.065734,0.015302,0.046548,0.066953,0.123176,0.022175,NaN,NaN,NaN
1217,0.315324,0.300839,0.316399,0.309968,0.294540,0.275474,0.294708,0.285094,0.076923,0.013003,0.023013,0.041642,0.112216,NaN,NaN,NaN,NaN
1218,0.309968,0.293825,0.308703,0.301103,0.285920,0.271961,0.289689,0.288712,0.146853,-0.008613,0.020767,0.033867,0.106300,NaN,NaN,NaN,NaN
1219,0.301103,0.295346,0.312279,0.308349,0.290948,0.274069,0.297576,0.289436,0.039161,-0.022175,0.006256,0.019074,0.089362,NaN,NaN,NaN,NaN


In [75]:
#interoperation of NA value
#df2 = df2.fillna(method='backfill')
#df2.head(5)

# just remove NA rows may be better method
df2 = df2.dropna(axis = 0)
df2.head(5)

,btcbegin,btchigh,btclow,btcend,cboebegin,cboehigh,cboelow,cboeend,cboevolumn,3hbefore,12hbefore,1dbefore,2dbefore,3hafter,12hafter,1dafter,2dafter
48,0.841661,0.852194,0.852028,0.847339,0.881466,0.855938,0.889861,0.892185,0.001399,-0.010144,0.024177,0.007462,0.190829,-0.014522,0.035133,0.037022,0.054471
49,0.847584,0.849842,0.850851,0.834259,0.881466,0.862966,0.889861,0.899421,0.002797,-0.003432,0.041376,0.028242,0.200015,0.017096,0.036084,0.063661,0.054926
50,0.834259,0.851696,0.859213,0.856183,0.892960,0.861560,0.901334,0.897974,0.001399,-0.002975,0.027527,0.029735,0.153075,0.041680,0.014158,0.057667,0.053269
51,0.856183,0.846330,0.847294,0.830488,0.881466,0.848911,0.879105,0.874096,0.002797,0.014522,0.051364,0.052252,0.147952,0.056754,0.024044,0.067292,0.039494
52,0.830488,0.831418,0.800146,0.792579,0.870690,0.838370,0.864764,0.859624,0.002797,-0.017096,-0.012397,0.012077,0.069803,0.026410,-0.002783,0.054590,0.003923


In [76]:
df2.tail(5)

,btcbegin,btchigh,btclow,btcend,cboebegin,cboehigh,cboelow,cboeend,cboevolumn,3hbefore,12hbefore,1dbefore,2dbefore,3hafter,12hafter,1dafter,2dafter
1167,0.195842,0.184845,0.204088,0.200102,0.183908,0.173577,0.193604,0.185962,0.092308,0.002536,0.000506,-0.024782,0.028472,-0.007826,-0.043570,-0.058991,-0.122739
1168,0.200102,0.189146,0.208410,0.203109,0.188218,0.174280,0.196472,0.186686,0.078322,0.007229,0.003432,-0.007786,0.022368,-0.011639,-0.043280,-0.056223,-0.123176
1169,0.203109,0.188253,0.208198,0.203788,0.188937,0.182713,0.197189,0.198987,0.127273,0.012805,0.006236,-0.001946,0.011234,-0.011437,-0.036375,-0.070573,-0.112216
1170,0.203668,0.197393,0.212357,0.211615,0.200431,0.183415,0.202926,0.193198,0.165035,0.007826,0.014522,-0.005084,0.015717,-0.007084,-0.048136,-0.072433,-0.106300
1171,0.211741,0.199571,0.216191,0.214544,0.194684,0.184821,0.204360,0.198987,0.113287,0.011639,0.018334,0.016333,0.023386,-0.006602,-0.035407,-0.070288,-0.089362


In [77]:
tmp1 = (df2['3hbefore'] < 0.0001)*1
df2['tmp1'] = tmp1

In [78]:
df2.head(5)

,btcbegin,btchigh,btclow,btcend,cboebegin,cboehigh,cboelow,cboeend,cboevolumn,3hbefore,12hbefore,1dbefore,2dbefore,3hafter,12hafter,1dafter,2dafter,tmp1
48,0.841661,0.852194,0.852028,0.847339,0.881466,0.855938,0.889861,0.892185,0.001399,-0.010144,0.024177,0.007462,0.190829,-0.014522,0.035133,0.037022,0.054471,1
49,0.847584,0.849842,0.850851,0.834259,0.881466,0.862966,0.889861,0.899421,0.002797,-0.003432,0.041376,0.028242,0.200015,0.017096,0.036084,0.063661,0.054926,1
50,0.834259,0.851696,0.859213,0.856183,0.892960,0.861560,0.901334,0.897974,0.001399,-0.002975,0.027527,0.029735,0.153075,0.041680,0.014158,0.057667,0.053269,1
51,0.856183,0.846330,0.847294,0.830488,0.881466,0.848911,0.879105,0.874096,0.002797,0.014522,0.051364,0.052252,0.147952,0.056754,0.024044,0.067292,0.039494,0
52,0.830488,0.831418,0.800146,0.792579,0.870690,0.838370,0.864764,0.859624,0.002797,-0.017096,-0.012397,0.012077,0.069803,0.026410,-0.002783,0.054590,0.003923,1


In [79]:
 #change value to 0/1 flag
tmp3h = (df2['3hbefore'] < 0.0001)*1
df2['3hbefore'] = tmp3h

tmp12h = (df2['12hbefore'] < 0.0001)*1
df2['12hbefore'] = tmp12h

tmp1d = (df2['1dbefore'] < 0.0001)*1
df2['1dbefore'] = tmp1d

tmp2d = (df2['2dbefore'] < 0.0001)*1
df2['2dbefore'] = tmp2d

df2.head(5)

,btcbegin,btchigh,btclow,btcend,cboebegin,cboehigh,cboelow,cboeend,cboevolumn,3hbefore,12hbefore,1dbefore,2dbefore,3hafter,12hafter,1dafter,2dafter,tmp1
48,0.841661,0.852194,0.852028,0.847339,0.881466,0.855938,0.889861,0.892185,0.001399,1,0,0,0,-0.014522,0.035133,0.037022,0.054471,1
49,0.847584,0.849842,0.850851,0.834259,0.881466,0.862966,0.889861,0.899421,0.002797,1,0,0,0,0.017096,0.036084,0.063661,0.054926,1
50,0.834259,0.851696,0.859213,0.856183,0.892960,0.861560,0.901334,0.897974,0.001399,1,0,0,0,0.041680,0.014158,0.057667,0.053269,1
51,0.856183,0.846330,0.847294,0.830488,0.881466,0.848911,0.879105,0.874096,0.002797,0,0,0,0,0.056754,0.024044,0.067292,0.039494,0
52,0.830488,0.831418,0.800146,0.792579,0.870690,0.838370,0.864764,0.859624,0.002797,1,1,0,0,0.026410,-0.002783,0.054590,0.003923,1


In [ ]:
tmp3hafter = (df2['3hafter'] < 0.0001)*1
df2['3hafter'] = tmp3hafter

df2['12hafter'] = (df2['12hafter'] < 0.0001)*1
df2['1dafter'] = (df2['1dafter'] < 0.0001)*1
df2['2dafter'] = (df2['2dafter'] < 0.0001)*1

In [30]:
#interoperation of NA value
#df2 = df2.fillna(method='backfill')
#df2.head(5)

# just remove NA rows may be better method

,btcbegin,btchigh,btclow,btcend,cboebegin,cboehigh,cboelow,cboeend,cboevolumn,3hbefore
0,0.650832,0.674454,0.659420,0.647569,0.619971,0.593113,0.628854,0.621563,0.000000,0.057398
1,0.647569,0.729524,0.667486,0.681184,0.681753,0.666550,0.690521,0.697178,0.000699,0.057398
2,0.681184,0.701684,0.699340,0.708230,0.743534,0.739986,0.752187,0.772793,0.001399,0.057398
3,0.708230,0.749311,0.729855,0.760685,0.784483,0.772312,0.793059,0.806078,0.002797,0.057398
4,0.760685,0.773711,0.771630,0.785993,0.823276,0.870696,0.821741,0.907381,0.008392,0.113115


In [41]:
#change value to 0/1 flag
df2['3hflag'] = (df2['3hbefore'] < 0.0000001)*1
df2


,btcbegin,btchigh,btclow,btcend,cboebegin,cboehigh,cboelow,cboeend,cboevolumn,3hbefore,3hflag
0,0.650832,0.674454,0.659420,0.647569,0.619971,0.593113,0.628854,0.621563,0.000000,0.057398,0
1,0.647569,0.729524,0.667486,0.681184,0.681753,0.666550,0.690521,0.697178,0.000699,0.057398,0
2,0.681184,0.701684,0.699340,0.708230,0.743534,0.739986,0.752187,0.772793,0.001399,0.057398,0
3,0.708230,0.749311,0.729855,0.760685,0.784483,0.772312,0.793059,0.806078,0.002797,0.057398,0
4,0.760685,0.773711,0.771630,0.785993,0.823276,0.870696,0.821741,0.907381,0.008392,0.113115,0
5,0.785993,0.803718,0.801322,0.778746,0.919540,0.886156,0.881973,0.876990,0.011189,0.104808,0
6,0.778746,0.785980,0.780026,0.773694,0.869971,0.841883,0.874086,0.877713,0.002797,0.070515,0
7,0.773438,0.784019,0.795327,0.787141,0.874282,0.917779,0.882690,0.888567,0.019580,0.012753,0
8,0.787141,0.803911,0.805774,0.808378,0.916667,0.883345,0.924996,0.920405,0.000000,0.001148,0
9,0.808805,0.804812,0.818792,0.808924,0.933908,0.903022,0.940054,0.938495,0.006993,0.030059,0
